# PyTorch Workflow

Resources:
- notebook: https://github.com/mrdbourke/pytorch-deep-learning/blob/main/01_pytorch_workflow.ipynb
- website: https://www.learnpytorch.io/01_pytorch_workflow/
- questions: https://github.com/mrdbourke/pytorch-deep-learning/discussions

In [ ]:
what_were_covering = {1: 'data covering into tensors (prepare and load)',
                      2: 'build or pick model',
                      3: 'fitting model to data (training)',
                      4: 'making predictions and evaluating a model',
                      5: 'saving and loading model',
                      6: 'combining everything and saving model to reload'}

what_were_covering

In [ ]:
import torch
from torch import nn # nn = neutral network, PyTorch's building blocks
import matplotlib.pyplot as plt
import numpy as np

# check PyTorch version

torch.__version__

### 1. Preparing and loading data

data can be almost everything, what we can present in numeric representation, in Machine Learning

For example:
- Excel spreadsheet
- Images of any kind
- Videos, Youtube
- Audio, like songs or podcasts
- Text
- Dna

Machine learning is a game of two parts:
1. Get data into a numerical representation
2. Build a model to learn patterns in numerical representation

In [ ]:
### Creating data

# Create *known* parameters
# linear function formula: Y = a + bX; a = bias, b = weight

weight = 0.7 #waga, siła połączeń między neuronami
bias = 0.3  #błąd systematyczny

# Create data
start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1) # tensor, matrix
y = weight * X + bias #linear function

# show data
print(f"X tensor (first ten):\n {X[:10]}")
print(f"y tensor (first ten):\n {y[:10]}")
print(f"length of X: {len(X)} and len of y: {len(y)}")

### Splitting data into training and test sets, what is one of the most important concepts in Machine Learning

In [ ]:
# Create a train/test split
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(X_test)

Visualizating data

In [ ]:
def plot_predictions(train_data = X_train,
                     train_labels = y_train,
                     test_data = X_test,
                     test_labels = y_test,
                     predictions = None):
  """
  Plots training data
  """
  plt.figure(figsize=(10,7))

  # Plot training data in blue
  plt.scatter(train_data, train_labels, c='b', s=4, label='Trainining data')

  # Plot test data in green
  plt.scatter(test_data, test_labels, c='g', s=4, label='Testing data')

  # Are there predictions?
  if predictions is not None:
    # Plot predictions
    plt.scatter(test_data, predictions, c='r', s=4, label='Predictions')

  # Show the legend
  plt.legend(prop={"size": 14})

In [ ]:
plot_predictions()

### Building PyTorch model

OOP: https://realpython.com/python3-object-oriented-programming/

What Model does:
- start with random values of weight and bias
- Check training data and adjust the random values to get closer to ideal values

How does it do so:
two main algorithms:
1. gradient descent: https://www.youtube.com/watch?v=IHZwWFHWa-w
2. backpropagation: https://www.youtube.com/watch?v=Ilg3gGewQ5U


In [ ]:
from torch import nn

# Create linear regression model class
class LinearRegressionModel(nn.Module): # nn.Module is lego building blocks for PyTorch
  def __init__(self):
    super().__init__()
    self.weights = nn.Parameter(torch.randn(1,
                                           requires_grad = True,
                                           dtype = torch.float))
    self.bias = nn.Parameter(torch.randn(1,
                                         requires_grad = True,
                                         dtype=torch.float))

  def forward(self, x: torch.Tensor) -> torch.Tensor: # "x" is the input data
    return self.weights * x + self.bias


from sys import modules
### PyTorch model building essentials

* torch.nn - contains all of the buildings for computational graphs (neural network)
* torch.nn.Parameter - what parameters should our model try and learn
* torch.nn.Module - the bvase class for all neural network modules
* torch.optim - optimalize parameters to pronouce better results
* def forward() - All nn.Module subclasses require to overwrite forward(), this emtod defines what happens in the forward computation
* torch.util.data.Dataset -
* torch.utils.data.DataLoader -

In [ ]:
# What's inside our model

# we can check our model parameters or what's inside by using '.parameters()'

# random seed
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)

# instance of the model
model_0 = LinearRegressionModel()

# check parameters
list(model_0.parameters())

In [ ]:
# List named parameters
model_0.state_dict()

In [ ]:
# goal is to get close to this values
weight, bias

### Making predictions with our model with 'torch.inference_mode()'

To check our model's predictive power, let's see how well it predicts y_test based on X_test

When we pass data through model, it's going to run it thorugh the forward() method.

In [ ]:
# Make predictions
with torch.inference_mode():
  y_preds = model_0(X_test)

y_preds

In [ ]:
plot_predictions(predictions = y_preds)

## Training model

the whole idea of training model is to evelop random parameters into parameters, which are as close as they can to *known* parameters

One way to measure how poor or how bad our models predictions are is to use pytorch loss functions

* Note: Loss function may also be called cost function or criterion in different areas

Thing we need to train:

* **Loss functions:** A function to measure how wrong model's predictions are

* **Optimizer** takes into account the loss of a model and adjust the model's parameters (e.g weight and bias in our case) to improve the loss function

For PyTorch we need:
* A training loop
* A testing loop

In [ ]:
# Setup a loss function
loss_fn = nn.L1Loss()

# Setup an optimizer
optimizer = torch.optim.SGD(params = model_0.parameters(),
                            lr = 0.01) # lr = learning rate = possibly the most important hyperparameter you can set

In [ ]:
loss_fn

### Building a training loop (and a testing loop) in PyTorch

A couple of things we need in a training loop:
0. Loop through data and do:
1. Forward pass (this involves data moving through our model's forward()' function) to make predictions on data
2. calculate the loss (compare forward pass predictions to ground truth labels)
3. Optimizer zero grad
4. Loss backward - move backwards through the network to calculate the gradients of each of the parameters of our model with respect to the loss
5. Optimizer step - use the optimizer to adjust our model's parameters to try and improve the loss (**gradient descent**)

In [ ]:
model_0.state_dict()

In [ ]:
# An epoch is one loop thorugh the data
epochs = 200
torch.manual_seed(RANDOM_SEED)

# Track different values with tracking progress
epoch_count = []
loss_values = []
test_loss_values = []
#Training
# 0. loop through data for a number of epochs
for epoch in range(epochs):
  # Song:
  # It's train time!
  # Do the forward pass,
  # Calculate the loss
  # optimizer zero grad,
  # loss backwards
  #
  # Optimizer step step step
  #
  # Let's test now!
  # With torch no grad:
  # do the forward pass
  # calculate the loss
  # watch it go down down down

  model_0.train() # train mode sets all parameters that require gradients to require gradients

  # 1. Forward pass with training data
  y_pred = model_0.forward(X_train)

  # 2. Calculate the loss (how our model's prediction are wrong to correct answears)
  loss = loss_fn(y_pred, y_train)
  # 3. Optimimizer zero grad
  optimizer.zero_grad() # reset grad ( beacouse grad accumulate after a few loops )

  # 4. Loss backward (backpropagation na bazie grad dla każdego elementu; zmienia potencjalnie wagi lub bias, aby zblizyc sie do docelowego inputu)
  loss.backward()

  # 5. Step the optimizer
  optimizer.step() # taking a "step" to update values of our parameters to perform closer output

  # model_0.eval() - wyłącza śledzenie gradientu
  model_0.eval() # wyłącza niepotrzebne funkcje
  with torch.inference_mode(): # turns off gradient tracking & a couple more things
    # 1 forward mode
    test_pred = model_0(X_test)

    # 2. Calculate the loss
    test_loss = loss_fn(test_pred, y_test)

  if epoch % 10 == 0:
    epoch_count.append(epoch)
    loss_values.append(loss)
    test_loss_values.append(test_loss)
    print(f"Epoch: {epoch} | loss: {loss} | Test loss: {test_loss}")



In [ ]:
# plot the loss curves
plt.plot(epoch_count, np.array(torch.tensor(loss_values)),label = 'Loss value')
plt.plot(epoch_count, test_loss_values, label='Test loss value')
plt.legend()
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.title('Training and test loss accuracy')

In [ ]:
model_0.state_dict()

In [ ]:
weight, bias

In [ ]:
with torch.inference_mode():
  y_pred_new = model_0(X_test)


In [ ]:
plot_predictions(predictions = y_preds)

In [ ]:
plot_predictions(predictions = y_pred_new)

### Saving a model in PyTorch

There are three main methods you should know about for saving and loading models in PyTorch.

1. 'torch.save()' - allows to save a PyTorch object in Python pickle format
2. 'torch.load()' - allows to load a saved PyTorch object
3. 'torch.nn.Module.load_state_dict()' - allows to load a model's saved state dictionary; to simplify that loads parameters with "weights"


In [ ]:
from pathlib import Path


# 1. Creating model directory
MODEL_DIRECTORY = Path("Models")
MODEL_DIRECTORY.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME = "01_pytorch_model.pth"
MODEL_SAVING = MODEL_DIRECTORY / MODEL_NAME

# 3. Save the model state dict
print(f"Model saved to {MODEL_DIRECTORY}/{MODEL_NAME}")
torch.save(obj=model_0.state_dict(),
           f=MODEL_SAVING)

In [ ]:
# Loading the model from saved copy

# 1. Creating empty model class
loaded_model = LinearRegressionModel()

# 2. Loading model state dict from saved model directory
loaded_model.load_state_dict(torch.load(MODEL_SAVING))

# Checking if is everything all right loaded
loaded_model.state_dict()

In [ ]:
# Make some predictions:
loaded_model.eval()
with torch.inference_mode():
  loaded_model_pred = loaded_model(X_test)
np.array(torch.tensor(loaded_model_pred))
plot_predictions(predictions= np.array(torch.tensor(loaded_model_pred)))

In [ ]:
# Compare loaded model preds with original model

loaded_model_pred == y_pred_new

## Puttint it all together


### 6.1 Data

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Create device agnostic code
device_destination = 'cuda' if torch.cuda.is_available() else 'cpu'
device_destination

In [ ]:
## Create data with random state

# Create known values as weight and bias
weight = 0.7
bias = 0.3

# Data
RANDOM_STATE = 42
torch.manual_seed(RANDOM_STATE)

# number settings
start = 0
end = 1.0
step = 0.01

# creating random numbers for data
x = torch.arange(start= start, end= end, step = step).unsqueeze(dim=1)
y = weight * x + bias # y = wx + b

# summing up what we have done
print(f"Our x ranges from {start} to {end} with {step} step")
print(f"length of x: {len(x)} | length of y: {len(y)}")

In [ ]:
## Create train and test data
train_percentage = 80
portion = int((train_percentage) * 0.01 * len(x))

X_train, y_train, X_test, y_test = x[:portion], y[:portion], x[portion:], y[portion:]

print(f"our dataset was splitted to {train_percentage}% train and {100 - train_percentage}% test")
print(f"train size: {len(X_train)} | test size: {len(X_test)}")

In [ ]:
# Create model
class LinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()

    self.linear_layer = nn.Linear(in_features=1,
                                  out_features=1)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    return  self.linear_layer(x)

In [ ]:
torch.manual_seed(RANDOM_STATE)
module_01 = LinearRegressionModel()
module_01.state_dict()

In [ ]:
next(module_01.parameters()).device

In [ ]:
module_01.to(device_destination)
next(module_01.parameters()).device

In [ ]:
# Create pyplot visualization function

def plot_prediction(X_train = X_train,
                    y_train = y_train,
                    X_test = X_test,
                    y_pred = None):
  plt.figure(figsize=(10, 7))
  plt.scatter(X_train, y_train, s=10, label = 'Train data')
  plt.scatter(X_test, y_test, s=10, label = 'Test data')

  if not (y_pred == None):
    plt.scatter(X_test, y_pred, s=10, label = 'Predictions')

  plt.legend()
  plt.title('Model predictions compared to test and train data')

In [ ]:
plot_prediction()

In [ ]:
loss_fn = nn.L1Loss()

In [ ]:
lr = 0.01
optimizer = torch.optim.SGD(params = module_01.parameters(), lr = lr)

In [ ]:
print(f"X_train device: {X_train.device} | y_train device: {y_train.device} | module_01 device: {module_01.parameters}")

In [ ]:
# training
torch.manual_seed(RANDOM_STATE)

# Put data on correct device
X_train = X_train.to(device_destination)
X_test = X_test.to(device_destination)
y_train = y_train.to(device_destination)
y_test = y_test.to(device_destination)


epochs = 200
for epoch in range(epochs):
  module_01.train()

  # Forward pass
  y_pred = module_01(X_train)

  # Calculate the loss
  loss = loss_fn(y_pred, y_train)

  # Optimizer zero grad
  optimizer.zero_grad()

  # loss backward
  loss.backward()

  # Optimizer step step step
  optimizer.step()

  ### Testing
  module_01.eval()

  with torch.inference_mode():
    test_pred = module_01.forward(X_test)

    test_loss = loss_fn(test_pred, y_test)

  # Print out

  if epoch % 10 == 0:
    print(f"epoch: {epoch} | loss: {loss} | test_loss: {test_loss}")


In [ ]:
module_01.state_dict()

In [ ]:
print(y_pred.device)
print(X_test.device)
print(y_test.device)
print(X_train.device)
print(y_train.device)

In [ ]:
def to_cpu(*args: torch.Tensor) -> torch.Tensor:
  args = args.to('cpu')
  return args

In [ ]:
# y_preds.to('cuda')

In [ ]:
module_01.eval()

with torch.inference_mode():
  y_preds = module_01(X_test)

y_preds = y_preds.to('cpu')
X_train = X_train.to('cpu')
y_train = y_train.to('cpu')
X_test = X_test.to('cpu')
y_test = y_test.to('cpu')

In [ ]:
plot_prediction(y_pred = y_preds)

In [ ]:
# Saving model
from pathlib import Path
PATH = Path('Models')
PATH.mkdir(parents=True, exist_ok=True)

NAME = 'Module_01.saved_version_1.pt'
DIRECTORY_PATH = PATH / NAME
torch.save(module_01.state_dict(), DIRECTORY_PATH)

In [ ]:
module_01_loaded = LinearRegressionModel()

module_01_loaded.load_state_dict(torch.load(DIRECTORY_PATH))
module_01_loaded.state_dict()
module_01_loaded.to(device_destination)

In [ ]:
module_01_loaded.eval()

with torch.inference_mode():
  y_pred_loaded = module_01_loaded(X_test)
  plot_prediction(X_train, y_train, X_test, y_pred_loaded)

In [ ]:
y_pred_loaded == y_preds

## Excercuses & Extra-curriculum

for excercise & extra-curriculum we can use:
  - https://www.learnpytorch.io/01_pytorch_workflow/#extra-curriculum
  - https://www.learnpytorch.io/01_pytorch_workflow/#exercises
  - https://github.com/mrdbourke/pytorch-deep-learning/tree/main/extras/exercises